### Define Required Libiraries 

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Import Dataset

In [3]:
#https://www.kaggle.com/ealaxi/paysim1?select=PS_20174392719_1491204439457_log.csv
df = pd.read_csv('PS_20174392719_1491204439457_log.csv')

In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


### Checking missing values in DataFrame

In [4]:
#Checking missing values in DataFrame. 
print(df.isnull().values.any())
print(df.isnull().sum().sum())

False
0


In [17]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

## EDA (Exploratory Data Analysis)


In [15]:
# Begining of Data wrangling
# There are the five types of transactions, however fraud occurs only in two of them TRANSFERs and CASH_OUTs
#the number of fraudulent TRANSFERs almost equals the number of fraudulent CASH_OUTs 

In [19]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [20]:
df.shape

(6362620, 11)

In [5]:
from collections import Counter
F = df['type']
F.count()
print(Counter(F))

#another Way
print(df.groupby(['type'])['type'].count())

Counter({'CASH_OUT': 2237500, 'PAYMENT': 2151495, 'CASH_IN': 1399284, 'TRANSFER': 532909, 'DEBIT': 41432})
type
CASH_IN     1399284
CASH_OUT    2237500
DEBIT         41432
PAYMENT     2151495
TRANSFER     532909
Name: type, dtype: int64


In [18]:
print(df.isFraud.value_counts())

0    6354407
1       8213
Name: isFraud, dtype: int64


In [6]:
print(df.loc[df.isFraud == 1].type.unique())   # If Fraud occurs, we need to the type of transaction

# Another Way
print('\n The types of fraudulent transactions are {}'.format(df.loc[df.isFraud == 1].type.drop_duplicates().values))
# So fraud occurs only in two of them TRANSFERs and CASH_OUTs

['TRANSFER' 'CASH_OUT']

 The types of fraudulent transactions are ['TRANSFER' 'CASH_OUT']


In [7]:
Fraud_Transfer_df = df.loc[(df.isFraud == 1) & (df.type == 'TRANSFER')]
print ('\n The number of fraudulent TRANSFERs = {}'.\
       format(len(Fraud_Transfer_df)))


Fraud_Cashout_df = df.loc[(df.isFraud == 1) & (df.type == 'CASH_OUT')]
print ('\n The number of fraudulent CASH_OUTs = {}'.\
       format(len(Fraud_Cashout_df))) 

##We can see that the number of fraudulent TRANSFERs almost equals the number of fraudulent CASH_OUTs 

# another way by counter
# F = df[df['type'] == 'TRANSFER']['isFraud']
# print('PAYMENT {}'.format(Counter(F)))


 The number of fraudulent TRANSFERs = 4097

 The number of fraudulent CASH_OUTs = 4116


In [92]:
# df.loc[(df.isFraud == 1)].type.str.contains('TRANSFER').count()

In [79]:
df.loc[(df.isFraud == 1) & (df.type == 'TRANSFER')].apply(len)
# len(df.loc[(df.isFraud == 1) & (df.type == 'TRANSFER')])

step              4097
type              4097
amount            4097
nameOrig          4097
oldbalanceOrg     4097
newbalanceOrig    4097
nameDest          4097
oldbalanceDest    4097
newbalanceDest    4097
isFraud           4097
isFlaggedFraud    4097
dtype: int64

In [81]:
print('\nThe type of transactions in which isFlaggedFraud is set: \
{}'.format(list(df.loc[df.isFlaggedFraud == 1].type.drop_duplicates()))) 


The type of transactions in which isFlaggedFraud is set: ['TRANSFER']


In [80]:
df.loc[df.isFlaggedFraud == 1].type.unique()

array(['TRANSFER'], dtype=object)

In [82]:
dfTransfer = df.loc[df.type == 'TRANSFER']
dfTransfer

# After many investigation, The threshold for isFlaggedFraud  is not clear , it is set for only 16 times,
# So we will discard it and make it as insignifcant feature

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.00,0.0,C553264065,0.00,0.00,1,0
19,1,TRANSFER,215310.30,C1670993182,705.00,0.0,C1100439041,22425.00,0.00,0,0
24,1,TRANSFER,311685.89,C1984094095,10835.00,0.0,C932583850,6267.00,2719172.89,0,0
58,1,TRANSFER,62610.80,C1976401987,79114.00,16503.2,C1937962514,517.00,8383.29,0,0
78,1,TRANSFER,42712.39,C283039401,10363.39,0.0,C1330106945,57901.66,24044.18,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362610,742,TRANSFER,63416.99,C778071008,63416.99,0.0,C1812552860,0.00,0.00,1,0
6362612,743,TRANSFER,1258818.82,C1531301470,1258818.82,0.0,C1470998563,0.00,0.00,1,0
6362614,743,TRANSFER,339682.13,C2013999242,339682.13,0.0,C1850423904,0.00,0.00,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1,0


In [ ]:
#Checking merchant  for cash in and cash out 

print('\nAre there any merchants among originator accounts for CASH_IN \
transactions? {}'.format(\
(df.loc[df.type == 'CASH_IN'].nameOrig.str.contains('M')).any())) # False

In [98]:
#https://www.geeksforgeeks.org/python-pandas-series-str-contains/#:~:text=Pandas%20Series.str.contains%20%28%29%20function%20is%20used%20to%20test,Index.%20Syntax%3A%20Series.str.contains%20%28pat%2C%20case%3DTrue%2C%20flags%3D0%2C%20na%3Dnan%2C%20regex%3DTrue%29

merchant_cashin = df.loc[(df.type == 'CASH_IN') & (df.nameOrig.str.contains('M'))]
len(merchant_cashin)

0

In [99]:
merchant_cashout = df.loc[(df.type == 'CASH_OUT') & (df.nameOrig.str.contains('M'))]
len(merchant_cashin)

0

So,no merchants for the originator accounts but Merchants are only included in destination accounts for all PAYMENTS.

### Data Cleaning

In [3]:
#fraud only occurs in 'TRANSFER's and 'CASH_OUT's, so we will drop unnecessary coumns

In [4]:
DfMl = df.loc[(df.type == 'TRANSFER') | (df.type == 'CASH_OUT')]
DfMl

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.00,0.0,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.0,C38997010,21182.00,0.00,1,0
15,1,CASH_OUT,229133.94,C905080434,15325.00,0.0,C476402209,5083.00,51513.44,0,0
19,1,TRANSFER,215310.30,C1670993182,705.00,0.0,C1100439041,22425.00,0.00,0,0
24,1,TRANSFER,311685.89,C1984094095,10835.00,0.0,C932583850,6267.00,2719172.89,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.0,C2080388513,0.00,0.00,1,0


In [5]:
DfMl2 = DfMl.drop(['nameOrig', 'isFraud' ,'nameDest', 'isFlaggedFraud'], axis = 1)

In [6]:
DfMl2

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
2,1,TRANSFER,181.00,181.00,0.0,0.00,0.00
3,1,CASH_OUT,181.00,181.00,0.0,21182.00,0.00
15,1,CASH_OUT,229133.94,15325.00,0.0,5083.00,51513.44
19,1,TRANSFER,215310.30,705.00,0.0,22425.00,0.00
24,1,TRANSFER,311685.89,10835.00,0.0,6267.00,2719172.89
...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,339682.13,0.0,0.00,339682.13
6362616,743,TRANSFER,6311409.28,6311409.28,0.0,0.00,0.00
6362617,743,CASH_OUT,6311409.28,6311409.28,0.0,68488.84,6379898.11
6362618,743,TRANSFER,850002.52,850002.52,0.0,0.00,0.00


In [7]:
# https://kanoki.org/2019/07/17/pandas-how-to-replace-values-based-on-conditions/

# DfMl2.applymap(str).replace('TRANSFER', 0)
# DfMl2.applymap(str).replace('CASH_OUT', 1)

In [7]:
DfMl2.loc[(DfMl2.type == 'TRANSFER'),'type']=0
DfMl2.loc[(DfMl2.type == 'CASH_OUT'),'type']= 1
DfMl2

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
2,1,0,181.00,181.00,0.0,0.00,0.00
3,1,1,181.00,181.00,0.0,21182.00,0.00
15,1,1,229133.94,15325.00,0.0,5083.00,51513.44
19,1,0,215310.30,705.00,0.0,22425.00,0.00
24,1,0,311685.89,10835.00,0.0,6267.00,2719172.89
...,...,...,...,...,...,...,...
6362615,743,1,339682.13,339682.13,0.0,0.00,339682.13
6362616,743,0,6311409.28,6311409.28,0.0,0.00,0.00
6362617,743,1,6311409.28,6311409.28,0.0,68488.84,6379898.11
6362618,743,0,850002.52,850002.52,0.0,0.00,0.00


In [8]:
# print(DfMl2.dtypes)
DfMl2.type = DfMl2.type.astype(int) # change to int
DfMl2.dtypes

step                int64
type                int32
amount            float64
oldbalanceOrg     float64
newbalanceOrig    float64
oldbalanceDest    float64
newbalanceDest    float64
dtype: object

In [9]:
IsFrad = DfMl['isFraud']
IsFrad

2          1
3          1
15         0
19         0
24         0
          ..
6362615    1
6362616    1
6362617    1
6362618    1
6362619    1
Name: isFraud, Length: 2770409, dtype: int64

In [10]:
YesFraddf = DfMl2.loc[IsFrad == 1] # we choose only YesFrad based on index in dataframe
YesFraddf

nonFrauddf = DfMl2.loc[IsFrad == 0]
nonFrauddf

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
15,1,1,229133.94,15325.00,0.0,5083.00,51513.44
19,1,0,215310.30,705.00,0.0,22425.00,0.00
24,1,0,311685.89,10835.00,0.0,6267.00,2719172.89
42,1,1,110414.71,26845.41,0.0,288800.00,2415.16
47,1,1,56953.90,1942.02,0.0,70253.00,64106.18
...,...,...,...,...,...,...,...
6362313,718,1,111963.72,4514.00,0.0,154925.27,266888.99
6362317,718,1,317177.48,170.00,0.0,345042.13,662219.61
6362320,718,1,159188.22,3859.00,0.0,0.00,159188.22
6362321,718,1,186273.84,168046.00,0.0,24893.67,211167.51


The data has multiple transactions with zero balances in the destination account both before and after (**a non-zero amount**) is transacted, so it seems there are missing values or so tricky fraud (low probablity). That is why we replace zero with -1 

In [11]:
DfMl2.loc[(DfMl2.oldbalanceDest == 0) & (DfMl2.newbalanceDest == 0) & (DfMl2.amount != 0), ['oldbalanceDest', 'newbalanceDest']] = - 1

In [12]:
DfMl2

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
2,1,0,181.00,181.00,0.0,-1.00,-1.00
3,1,1,181.00,181.00,0.0,21182.00,0.00
15,1,1,229133.94,15325.00,0.0,5083.00,51513.44
19,1,0,215310.30,705.00,0.0,22425.00,0.00
24,1,0,311685.89,10835.00,0.0,6267.00,2719172.89
...,...,...,...,...,...,...,...
6362615,743,1,339682.13,339682.13,0.0,0.00,339682.13
6362616,743,0,6311409.28,6311409.28,0.0,-1.00,-1.00
6362617,743,1,6311409.28,6311409.28,0.0,68488.84,6379898.11
6362618,743,0,850002.52,850002.52,0.0,-1.00,-1.00


In [ ]:
# the synthetic minority oversampling technique (SMOTE) contained in the 'imblearn' library. Motivated by this, I tried a variety of anomaly-detection and supervised learning approaches. I find, however, that the best result is obtained on the original dataset by using a ML algorithm based on ensembles of decision trees that intrinsically performs well on imbalanced data. Such algorithms not only allow for constructing a model that can cope with the missing values in our data, but they naturally allow for speedup via parallel-processing. Among these algorithms, the extreme gradient-boosted (XGBoost) algorithm used below slightly outperforms random-forest. Finally, XGBoost, like several other ML algorithms, allows for weighting the positive class more compared to the negative class --- a setting that also allows to account for the skew in the data.

In [17]:
X=DfMl2
Y= IsFrad

In [18]:
randomState = 3
np.random.seed(randomState)

In [19]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.3, random_state = randomState)

In [20]:
print(len(trainX))
print(len(testX))
print(len(trainY))

1939286
831123
1939286


https://xgboost.readthedocs.io/en/latest/parameter.html
https://xgboost.readthedocs.io/en/latest/python/python_api.html

**Parameter**

**scale_pos_weight** [default=1]:Control the balance of positive and negative weights, useful for unbalanced classes. A typical value to consider: sum(negative instances) / sum(positive instances)

**max_depth** [default=6]:Maximum depth of a tree. Increasing this value will make the model more complex and more likely to 
overfit.

**n_jobs** (Optional[int]) – Number of parallel threads used to run xgboost. When used with other Scikit-Learn algorithms like grid search, you may choose which algorithm to parallelize and balance the threads

In [21]:
weights = (Y == 0).sum() / ( (Y == 1).sum())  #  the balance of positive and negative weights
weights

336.3199805186899

In [22]:
XGBClassifiermodel = XGBClassifier(max_depth = 4, scale_pos_weight = weights, n_jobs = 4)

In [23]:
XGBClassifiermodel

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=4,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=4, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=336.3199805186899,
              subsample=None, tree_method=None, use_label_encoder=True,
              validate_parameters=None, verbosity=None)

In [24]:
ModelRe = XGBClassifiermodel.fit(trainX, trainY).predict_proba(testX)

C:\Users\Ahmed\ana\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:21:56] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [25]:
ModelRe

array([[9.9988872e-01, 1.1129369e-04],
       [9.9999195e-01, 8.0349491e-06],
       [9.9992490e-01, 7.5120515e-05],
       ...,
       [9.9999934e-01, 6.6894671e-07],
       [9.9999171e-01, 8.3002960e-06],
       [9.9999577e-01, 4.2046977e-06]], dtype=float32)

In [26]:
print('precision = {}'.format(average_precision_score(testY, ModelRe[:, 1])))

# not high precision 

precision = 0.9624498206196488


In [ ]:
# pip install pygraphviz

In [29]:
to_graphviz(XGBClassifiermodel)


In [ ]:
from xgboost import plot_tree
import matplotlib.pyplot as plt
plot_tree(XGBClassifiermodel)
plt.show()